In [49]:
from PIL import Image, ImageFont, ImageDraw, ImageOps
import numpy as np
from matplotlib import pyplot as plt
import csv

In [50]:
string = '𐎀𐎁𐎂𐎃𐎄𐎅𐎆𐎇𐎈𐎉𐎊𐎋𐎌𐎍𐎎𐎏𐎐𐎑𐎒𐎓𐎔𐎕𐎖𐎗𐎘𐎙𐎚𐎛𐎜𐎝'.upper()
alphabet = []
for letter in string:
    alphabet.append(letter)
print(alphabet)

['𐎀', '𐎁', '𐎂', '𐎃', '𐎄', '𐎅', '𐎆', '𐎇', '𐎈', '𐎉', '𐎊', '𐎋', '𐎌', '𐎍', '𐎎', '𐎏', '𐎐', '𐎑', '𐎒', '𐎓', '𐎔', '𐎕', '𐎖', '𐎗', '𐎘', '𐎙', '𐎚', '𐎛', '𐎜', '𐎝']


In [51]:
def simple_binarization(image, threshold):  #просто функция для бинаризации 
    img_arr = np.array(image)
    new_image = np.zeros(shape=img_arr.shape)
    new_image[img_arr > threshold] = 255
    return Image.fromarray(new_image.astype(np.uint8), 'L')

In [52]:
def cut_white_image_parts(image):  #проходимся по 4 направлениям и ищем границы 

    for x in range(image.width-1, -1, -1):
        is_col_empty = True
        for y in range(image.height):
            if image.getpixel((x, y)) < 255:
                is_col_empty = False
                break
#выполняем до тех пор пока не найдем непустую строку или столбец
        if not is_col_empty:
            right_pixel = x
            break
            
    for y in range(image.height-1, -1, -1):
        is_row_empty = True
        for x in range(image.width):
            if image.getpixel((x, y)) < 255:
                is_row_empty = False
                break

        if not is_row_empty:
            bottom_pixel = y
            break
    
    for x in range(0, image.width):
        is_col_empty = True
        for y in range(image.height):
            if image.getpixel((x, y)) < 255:
                is_col_empty = False
                break

        if not is_col_empty:
            left_pixel = x
            break
            
    for y in range(0, image.height):
        is_row_empty = True
        for x in range(image.width):
            if image.getpixel((x, y)) < 255:
                is_row_empty = False
                break

        if not is_row_empty:
            upper_pixel = y
            break
            

    return image.crop(box=(left_pixel, upper_pixel, right_pixel, bottom_pixel)) #изображение обрезанное по границам

In [53]:
def cut_empty_rows_and_cols(image):

    empty_row_numbers = []
    empty_column_numbers = []

    for x in range(image.width): #ищем пустые строки и столбцы 
        is_col_empty = True
        for y in range(image.height):
            if image.getpixel((x, y)) < 255:
                is_col_empty = False
                break

        if is_col_empty:
            empty_column_numbers.append(x)

    for y in range(image.height):
        is_row_empty = True
        for x in range(image.width):
            if image.getpixel((x, y)) < 255:
                is_row_empty = False
                break

        if is_row_empty:
            empty_row_numbers.append(y)

    def last_element_in_a_row(elements, start_element, step):  #определяем края обрезки
        prev_element = start_element

        for element in elements[::step]:
            if abs(element - prev_element) > 1:
                return prev_element + step

            prev_element = element

        return prev_element + step

    left_whitespace_end = last_element_in_a_row(empty_column_numbers, -1, 1)
    upper_whitespace_end = last_element_in_a_row(empty_row_numbers, -1, 1)
    right_whitespace_end = last_element_in_a_row(empty_column_numbers, image.width, -1)
    lower_whitespace_end = last_element_in_a_row(empty_row_numbers, image.height, -1)

    return image.crop(box=(left_whitespace_end, upper_whitespace_end, right_whitespace_end + 1, lower_whitespace_end + 1)) #обрезаем пустоту, оставляя просто прямоугольник

In [54]:
def font_generate():
    font_size = 150
    font = ImageFont.truetype("output/fonts/NotoSansUgaritic-Regular.ttf", font_size)

    for letter in alphabet:
        img = Image.new(mode="L", size=(font_size, font_size), color=255)
        draw = ImageDraw.Draw(img, mode = 'L')
        draw.text(xy=(0, 0), text=letter, fill=0, font=font, anchor = 'lt')
        cutted_img = cut_empty_rows_and_cols(img)
        threshloded_img = simple_binarization(cutted_img, 100)
        threshloded_img.save(f'output/letter_images/letter_{letter}.png')

In [55]:
def color_used_arr(img):  #преобразование в двумерный массив, где 1 - черный символ а 0 белый фон
    return np.asarray(np.asarray(img) < 1, dtype = np.int0)

def get_features(img):
    img_pixels = img.load()
    
    width = img.size[0]
    height = img.size[1]

    size = width * height

    weight, rel_weight, x_avg, y_avg, rel_x_avg, rel_y_avg = 0, 0, 0, 0, 0, 0
    inertia_x, rel_inertia_x, inertia_y, rel_inertia_y = 0, 0, 0, 0

    for i in range(width):   #считаем вес
        for j in range(height):
            if img_pixels[i, j] == 0: #если черный, то ...
                weight += 1
                x_avg += i  # сумма для центра тяжести 
                y_avg += j

    rel_weight = weight / size    #относительный вес(на площадь)

    x_avg /= weight
    y_avg /= weight
    rel_x_avg = (x_avg - 1) / (width - 1)  #нормирование средних значений
    rel_y_avg = (y_avg - 1) / (height - 1)  #координаты центра тяжести, нормированные от 0 до 1(0 - край)

    for i in range(width):  #момент инерции (сумм кв расст от черных до центра тяжести)
        for j in range(height):
            if img_pixels[i, j] == 0: #если черный 
                inertia_x = (j - x_avg) ** 2
                inertia_y = (i - y_avg) ** 2

    rel_inertia_x = inertia_x / (width ** 2 + height ** 2)  #нормирование моментов инерции
    rel_inertia_y = inertia_y / (width ** 2 + height ** 2)
    
    return {
        'weight': weight,
        'rel_weight': rel_weight,
        'x_avg': x_avg,
        'y_avg': y_avg,
        'rel_x_avg': rel_x_avg,
        'rel_y_avg': rel_y_avg,
        'inertia_x': inertia_x,
        'inertia_y': inertia_y,
        'rel_inertia_x': rel_inertia_x,
        'rel_inertia_y': rel_inertia_y
    }

In [56]:
def get_profiles(img):
    img_arr_for_calculations = color_used_arr(img)
    
    x_profiles = np.sum(img_arr_for_calculations, axis=0)
    x_range = np.arange(1, img_arr_for_calculations.shape[1] + 1)
    
    y_profiles = np.sum(img_arr_for_calculations, axis=1)
    y_range = np.arange(1, img_arr_for_calculations.shape[0] + 1)  #суммирование черных по осям для получения профилей и их диапазонов значений
    
    return {
        'x_profiles': x_profiles,
        'x_range': x_range,
        'y_profiles': y_profiles,
        'y_range': y_range
    }

In [57]:
def write_profile(img, letter_path, type='x'):  #гистограммы
    profiles = get_profiles(img)
    
    if type == 'x':
        plt.bar(x=profiles['x_range'], height=profiles['x_profiles'], width=0.85)
        plt.ylim(0, max(profiles['x_profiles']))
        plt.xlim(0, max(profiles['x_range']))

    else:
        plt.barh(y=profiles['y_range'], width=profiles['y_profiles'], height=0.85)
        plt.ylim(max(profiles['y_range']), 0 )
        plt.xlim(0, max(profiles['y_profiles']))


    plt.savefig(letter_path)
    plt.clf()
    
    pass

In [58]:
def features_save():
    with open('ugarian_font.csv', 'w', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['letter', 'weight', 'rel_weight',
                                               'x_avg', 'y_avg', 'rel_x_avg', 'rel_y_avg', 
                                               'inertia_x', 'inertia_y', 'rel_inertia_x', 'rel_inertia_y'])
        writer.writeheader()

        for letter in alphabet:
            image = Image.open(f'output/letter_images/letter_{letter}.png')
            
            features = get_features(image)
            features['letter'] = letter
            
            writer.writerow(features)

In [59]:
def profile_save():
    for letter in alphabet:
        image = Image.open(f'output/letter_images/letter_{letter}.png')
        letter_x_profile_save_path = f'output/letter_profiles/letter_{letter}_x_profile.png'
        write_profile(image, letter_x_profile_save_path, 'x')
        letter_y_profile_save_path = f'output/letter_profiles/letter_{letter}_y_profile.png'
        write_profile(image, letter_y_profile_save_path, 'y')

In [60]:
font_generate()
features_save()
profile_save()

C:\Users\Kokovino\AppData\Local\Temp\ipykernel_9240\4012089895.py:2: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  return np.asarray(np.asarray(img) < 1, dtype = np.int0)


<Figure size 640x480 with 0 Axes>